In [ ]:
# STEP 1A: CHECK GPU AVAILABILITY
import subprocess

print('Checking GPU availability...\n')
try:
    gpu_info = subprocess.check_output(['nvidia-smi'], encoding='utf-8')
    print('✓ GPU detected! Training will be accelerated.')
    print('GPU Info:')
    print(gpu_info)
except:
    print('⚠ No GPU detected. Training will use CPU (slower).')
    print('\n💡 TIP: Enable GPU for 10x faster training:')
    print('   Runtime → Change runtime type → Hardware accelerator → T4 GPU')

In [ ]:
# STEP 1B: INSTALL REQUIRED PACKAGES
print('Checking package installation...\n')

# Always install in Colab to ensure correct versions
# This avoids import errors from incompatible versions
import sys

# Check if we're in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('🔧 Installing compatible packages for Colab...')
    print('   Installing MediaPipe, NumPy, and dependencies\n')
    
    # Install compatible versions
    # - mediapipe 0.10.13: Earliest available version in current PyPI
    # - numpy 1.26.4: Compatible with TF 2.19 and MediaPipe
    # - opencv-python: Ensure compatible version
    !pip install -q --upgrade pip
    !pip install -q 'mediapipe==0.10.13' 'numpy==1.26.4' 'opencv-python'
    !pip install -q tensorflowjs seaborn tqdm
    
    print('='*60)
    print('✓ Installation complete!')
    print('='*60)
    print('\n📦 Installed packages:')
    print('   • MediaPipe: 0.10.13')
    print('   • NumPy: 1.26.4')
    print('   • OpenCV: latest compatible')
    print('\n📋 NEXT STEPS:')
    print('   Continue running the remaining cells')
    print('='*60)
else:
    # Local environment - check if packages need installation
    needs_install = False
    install_reason = ""
    
    try:
        import mediapipe as mp
        import numpy as np
        
        # Check versions
        numpy_version = tuple(map(int, np.__version__.split('.')[:2]))
        
        # Verify compatible versions are installed
        if numpy_version >= (2, 0):
            needs_install = True
            install_reason = "NumPy 2.x detected (incompatible)"
        else:
            print('✓ Packages already installed')
            print(f'  • MediaPipe: {mp.__version__}')
            print(f'  • NumPy: {np.__version__}')
            print('  Skipping installation...\n')
            
    except ImportError as e:
        needs_install = True
        install_reason = "Packages not found"
        print(f'⚠ {install_reason}\n')
    
    if needs_install:
        if install_reason:
            print(f'⚠ Installation required: {install_reason}\n')
        
        print('Installing required packages...')
        !pip install -q mediapipe 'numpy==1.26.4' opencv-python
        !pip install -q tensorflowjs seaborn tqdm
        
        print('✓ Installation complete!\n')


In [ ]:
# STEP 1C: IMPORT ALL LIBRARIES
try:
    import os
    import json
    import sys
    import shutil
    import datetime
    import subprocess
    import numpy as np
    import cv2
    
    # Import MediaPipe with workaround for audio_classifier bug
    try:
        import mediapipe as mp
    except NameError as e:
        if 'audio_classifier' in str(e):
            # Workaround: Import only the vision solution we need
            print('⚠ MediaPipe audio module has import error (known bug)')
            print('  Applying workaround: importing vision solutions directly...\n')
            
            import mediapipe.python.solutions as solutions
            # Create a minimal mp object with just what we need
            class MediaPipeWrapper:
                solutions = solutions
            mp = MediaPipeWrapper()
        else:
            raise
    
    from pathlib import Path
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import confusion_matrix, classification_report
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from tensorflow.keras.regularizers import l2
    import matplotlib.pyplot as plt
    import seaborn as sns
    from tqdm import tqdm
    
    print('='*60)
    print('✓ All imports successful!')
    print('='*60)

except Exception as e:
    import traceback
    print('\n' + '='*60)
    print('⚠ Error During Import!')
    print('='*60)
    print(f'Error: {type(e).__name__}: {e}')
    print('\nFull traceback:')
    traceback.print_exc()
    print('\n📋 TROUBLESHOOTING:')
    print('1. Runtime → Restart runtime (CRITICAL - clears old imports)')
    print('2. Re-run Cell 1 (GPU check)')
    print('3. Re-run Cell 2 (package installation)')
    print('4. Then run this cell again')
    print('='*60)
    raise


In [ ]:
# STEP 1D: CONFIGURE TENSORFLOW FOR GPU/CPU
if tf.config.list_physical_devices('GPU'):
    print('Configuring TensorFlow for GPU acceleration...\n')
    
    # Enable mixed precision for 2-3x speedup (GPU only)
    from tensorflow.keras import mixed_precision
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print('✓ Mixed precision training enabled (float16)')
    
    # Enable GPU memory growth (prevents OOM errors)
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print('✓ GPU memory growth enabled')
    print('\n💡 GPU detected - Training will be 10x faster!')
else:
    print('⚠ Running on CPU - Training will take longer')
    print('\n💡 CPU MODE TIPS:')
    print('  • Expected training time: 30-60 minutes (vs 3-5 min on GPU)')
    print('  • Reduce batch size if you encounter memory errors')
    print('  • Consider using a smaller dataset for testing')
    print('  • GPU time limit in Colab: Enable GPU if you have quota available')

print(f"\n📦 Package Versions:")
print(f"  • TensorFlow: {tf.__version__}")
print(f"  • MediaPipe: {mp.__version__}")
print(f"  • NumPy: {np.__version__}")
print(f"  • OpenCV: {cv2.__version__}")

print(f"  • GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

print('\n' + '='*60)
print('✅ SETUP COMPLETE - Ready to proceed with training!')
print('='*60)

In [ ]:
# STEP 2: MOUNT GOOGLE DRIVE AND SET PATHS
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print('✓ Google Drive mounted successfully')
    IS_COLAB = True
except:
    print('⚠ Not running in Colab - using local paths')
    IS_COLAB = False

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()
print(f'✓ Random seeds set to {SEED}')

# Configure dataset source
# OPTION 1: Download from Kaggle (recommended for Colab)
USE_KAGGLE_DATASET = True  # Set to False if using your own dataset
KAGGLE_DATASET = 'kapillondhe/american-sign-language'

# OPTION 2: Use custom dataset path
CUSTOM_DATASET_PATH = "/content/drive/MyDrive/ASL_Dataset"  # Update if using custom dataset

# Output directory (always saved to Drive in Colab)
if IS_COLAB:
    OUTPUT_DIR = "/content/drive/MyDrive/asl_model_output"
else:
    OUTPUT_DIR = "asl_model_output"

print(f'\n📂 Output directory: {OUTPUT_DIR}')
print(f'\n⚠ DATASET SOURCE: {"Kaggle (auto-download)" if USE_KAGGLE_DATASET else "Custom path"}')

In [ ]:
# STEP 3: DOWNLOAD DATASET FROM KAGGLE (IF ENABLED)
if USE_KAGGLE_DATASET:
    print('Downloading dataset from Kaggle...')
    print(f'Dataset: {KAGGLE_DATASET}')
    
    # Install kagglehub if not already installed
    !pip install -q kagglehub
    
    import kagglehub
    from pathlib import Path
    
    try:
        # Download via kagglehub (handles authentication automatically in Colab)
        path = kagglehub.dataset_download(KAGGLE_DATASET)
        print(f'✓ kagglehub download complete: {path}')
        
        # Use the actual download path
        dataset_root = Path(path)
        
        # Check if ASL_Dataset subdirectory exists
        if (dataset_root / 'ASL_Dataset').exists():
            dataset_root = dataset_root / 'ASL_Dataset'
            print(f'✓ Found ASL_Dataset subdirectory')
        
        DATASET_PATH = str(dataset_root)
        print(f'✓ Using dataset from: {DATASET_PATH}')
        
    except Exception as e:
        print(f'❌ Error downloading dataset: {e}')
        print('⚠ Make sure you have internet connection in Colab')
        raise
else:
    # Use custom dataset path
    DATASET_PATH = CUSTOM_DATASET_PATH
    print(f'Using custom dataset path: {DATASET_PATH}')

# Validate dataset exists
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(
        f"Dataset path not found: {DATASET_PATH}\n"
        f"If using Kaggle: Check internet connection\n"
        f"If using custom: Update CUSTOM_DATASET_PATH in previous cell"
    )

# Get class names (letter folders only)
# Handle two dataset structures:
# Structure 1: ASL_Dataset/A/, ASL_Dataset/B/, ... (flat)
# Structure 2: ASL_Dataset/Train/A/, ASL_Dataset/Test/A/, ... (Train/Test split)
folders = sorted([d for d in os.listdir(DATASET_PATH) 
                  if os.path.isdir(os.path.join(DATASET_PATH, d))])

print(f"📂 Dataset structure detected: {folders[:5]}...")

# Check if Train/Test folders exist
if 'Train' in folders or 'Test' in folders:
    print("✓ Train/Test split structure detected")
    print("  Combining Train and Test folders for training...")
    
    # Use Train folder for getting class names (Test should have same classes)
    train_path = os.path.join(DATASET_PATH, 'Train')
    test_path = os.path.join(DATASET_PATH, 'Test')
    
    # Get class names from Train folder
    class_names = sorted([d for d in os.listdir(train_path) 
                          if os.path.isdir(os.path.join(train_path, d))])
    
    # Collect all image paths from both Train and Test
    all_image_paths = []
    class_image_counts = {}
    
    for class_name in class_names:
        class_image_counts[class_name] = 0
        
        # Get images from Train folder
        train_class_path = os.path.join(train_path, class_name)
        if os.path.exists(train_class_path):
            train_images = [os.path.join(train_class_path, f) 
                           for f in os.listdir(train_class_path) 
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            all_image_paths.extend([(img, class_name) for img in train_images])
            class_image_counts[class_name] += len(train_images)
        
        # Get images from Test folder
        test_class_path = os.path.join(test_path, class_name)
        if os.path.exists(test_class_path):
            test_images = [os.path.join(test_class_path, f) 
                          for f in os.listdir(test_class_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            all_image_paths.extend([(img, class_name) for img in test_images])
            class_image_counts[class_name] += len(test_images)
    
    # Store for later use in landmark extraction
    DATASET_STRUCTURE = 'train_test_split'
    DATASET_IMAGE_PATHS = all_image_paths
    total_images = len(all_image_paths)
    
else:
    print("✓ Flat structure detected (letter folders at root)")
    class_names = folders
    
    # Count images per class
    class_image_counts = {}
    for class_name in class_names:
        class_path = os.path.join(DATASET_PATH, class_name)
        count = len([f for f in os.listdir(class_path) 
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        class_image_counts[class_name] = count
    
    DATASET_STRUCTURE = 'flat'
    total_images = sum(class_image_counts.values())

num_classes = len(class_names)
print(f"\n✓ Found {num_classes} classes: {class_names}")

# Check if dataset includes non-letter classes (Nothing, Space, etc.)
non_letter_classes = [c for c in class_names if c not in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ']
if non_letter_classes:
    print(f"⚠ Non-letter classes detected: {non_letter_classes}")
    print(f"  These will be included in training for better model robustness")

# Validate class count (A-Z = 26, plus optional Nothing/Space/etc)
assert 24 <= num_classes <= 30, f"Unexpected class count: {num_classes}"
print(f"✓ Valid class count: {num_classes}")

print(f'\n✓ Total images across all classes: {total_images:,}')
print(f'\nSample counts per class:')
for class_name in sorted(class_image_counts.keys())[:5]:  # Show first 5
    print(f'  {class_name}: {class_image_counts[class_name]:,}')
if len(class_image_counts) > 5:
    print(f'  ... and {len(class_image_counts) - 5} more classes')

In [ ]:
# STEP 4: EXTRACT MEDIAPIPE HAND LANDMARKS
# Initialize MediaPipe Hands with optimized settings
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5
)

def extract_landmarks(image_path):
    """Extract 63 features from hand landmarks with wrist centering."""
    image = cv2.imread(image_path)
    if image is None:
        return None
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
        wrist = landmarks[0]
        landmarks_centered = landmarks - wrist  # Center at wrist (landmark 0)
        return landmarks_centered.flatten()
    
    return None

print("✓ MediaPipe landmark extractor ready")

# Process all images with progress tracking
X_data = []
y_labels = []
skipped_count = 0

print("\nExtracting landmarks from images...")
print("=" * 60)

# Use tqdm for better progress tracking in Colab
try:
    from tqdm import tqdm
    use_tqdm = True
except:
    use_tqdm = False
    print("Install tqdm for progress bars: !pip install tqdm")

# Handle different dataset structures
if DATASET_STRUCTURE == 'train_test_split':
    # Process pre-collected image paths from Train + Test folders
    print(f"Processing {total_images:,} images from Train and Test folders combined...")
    
    if use_tqdm:
        iterator = tqdm(DATASET_IMAGE_PATHS, desc="Extracting landmarks")
    else:
        iterator = DATASET_IMAGE_PATHS
    
    for img_path, class_name in iterator:
        landmarks = extract_landmarks(img_path)
        
        if landmarks is not None:
            X_data.append(landmarks)
            y_labels.append(class_name)
        else:
            skipped_count += 1
    
    print(f"✓ Processed all images from Train and Test folders")
    
else:
    # Process flat structure (original approach)
    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(DATASET_PATH, class_name)
        image_files = [f for f in os.listdir(class_path) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        print(f"[{class_idx+1}/{num_classes}] Processing class '{class_name}': {len(image_files)} images", end=' ')
        
        processed = 0
        if use_tqdm:
            iterator = tqdm(image_files, desc=f"Class {class_name}", leave=False)
        else:
            iterator = image_files
        
        for img_file in iterator:
            img_path = os.path.join(class_path, img_file)
            landmarks = extract_landmarks(img_path)
            
            if landmarks is not None:
                X_data.append(landmarks)
                y_labels.append(class_name)
                processed += 1
            else:
                skipped_count += 1
        
        print(f"→ {processed} successful, {len(image_files)-processed} skipped")

hands.close()

# Convert to numpy arrays
X_data = np.array(X_data, dtype=np.float32)
y_labels = np.array(y_labels)

skip_ratio = skipped_count / max(total_images, 1)
print(f"\n{'='*60}")
print(f"✓ Dataset loaded: {len(X_data)} samples")
print(f"✓ Feature shape: {X_data.shape}")
print(f"✓ Skipped (no hand detected): {skipped_count} ({skip_ratio:.1%})")
if skip_ratio > 0.2:
    print('⚠ WARNING: Over 20% images skipped - consider reviewing data quality')
print(f"{'='*60}")

# Check for NaN or Inf values (data quality check)
if np.isnan(X_data).any() or np.isinf(X_data).any():
    print('⚠ WARNING: NaN or Inf values detected in features!')
    print('Removing problematic samples...')
    valid_mask = ~(np.isnan(X_data).any(axis=1) | np.isinf(X_data).any(axis=1))
    X_data = X_data[valid_mask]
    y_labels = y_labels[valid_mask]
    print(f'✓ Cleaned dataset: {len(X_data)} samples remaining')

In [ ]:
# 5. LABEL ENCODING AND VERIFICATION
# Critical: Verify labels are ASL letters, not Train/Test
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_labels)

print("Label Mapping (index → class):")
print("="*40)
for i, label in enumerate(label_encoder.classes_):
    count = np.sum(y_labels == label)
    print(f"{i:2d} → {label:3s} ({count:5d} samples)")
print("="*40)

# Verify no incorrect labels
assert 'Train' not in label_encoder.classes_ and 'Test' not in label_encoder.classes_, \
    "CRITICAL ERROR: Train/Test detected in labels!"

# CRITICAL: Update num_classes to match actual labels found after extraction
# Some classes may have been lost during landmark extraction
num_classes = len(label_encoder.classes_)
print(f"\n✓ Training on {num_classes} classes (updated from label encoder)")
print(f"✓ Classes: {list(label_encoder.classes_)}")

In [ ]:
# STEP 6: STRATIFIED TRAIN/VAL SPLIT WITH DATA AUGMENTATION
# Stratified splits: 80% train, 20% validation
X_train, X_val, y_train, y_val = train_test_split(
    X_data, y_encoded, 
    test_size=0.2, 
    stratify=y_encoded,
    random_state=SEED
)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Input shape: {X_train.shape[1]} features")

# Verify label distribution
unique_train_labels = sorted(set(y_train))
print(f"\n✓ Training on class indices: {unique_train_labels}")
print(f"✓ Expected: 0 to {num_classes-1}")

# Data augmentation: Gaussian noise + horizontal flip
ENABLE_AUGMENTATION = True  # Set to False to disable augmentation

if ENABLE_AUGMENTATION:
    print('\nApplying data augmentation...')
    rng = np.random.default_rng(SEED)
    augmented_X, augmented_y = [], []
    
    for vec, label in zip(X_train, y_train):
        # Augmentation 1: Add Gaussian noise (σ=0.01)
        noise = vec + rng.normal(0, 0.01, size=vec.shape)
        augmented_X.append(noise.astype(np.float32))
        augmented_y.append(label)
        
        # Augmentation 2: Horizontal flip (mirror x-coordinates)
        mirrored = vec.copy()
        mirrored[0::3] *= -1  # Flip every 3rd element (x coordinates)
        augmented_X.append(mirrored.astype(np.float32))
        augmented_y.append(label)
    
    # Combine original + augmented
    X_train = np.concatenate([X_train, np.stack(augmented_X)], axis=0)
    y_train = np.concatenate([y_train, np.array(augmented_y)])
    
    print(f'✓ Augmentation complete: Train set expanded to {len(X_train):,} samples')
else:
    print('⚠ Augmentation disabled')

In [ ]:
# 7. MODEL ARCHITECTURE
# Critical: Output layer size must match num_classes (dynamically set)
def create_model(input_shape=63, num_classes=24):
    """Landmark-based classifier with dynamic output size.
    
    Best practices applied:
    - BatchNormalization for stable training
    - Dropout for regularization
    - He initialization for ReLU layers
    - L2 regularization to prevent overfitting
    """
    from tensorflow.keras.regularizers import l2
    
    model = keras.Sequential([
        layers.Input(shape=(input_shape,)),
        
        # First dense block
        layers.Dense(256, activation='relu', 
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # Second dense block
        layers.Dense(128, activation='relu',
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        # Third dense block (optional, improves capacity)
        layers.Dense(64, activation='relu',
                    kernel_initializer='he_normal'),
        layers.Dropout(0.1),
        
        # Output layer - DYNAMIC SIZE matching num_classes
        # Use dtype='float32' to ensure compatibility with mixed precision
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])
    return model

# Create and compile model with correct output size
model = create_model(input_shape=63, num_classes=num_classes)

# Use Adam optimizer with learning rate schedule
# Start with higher LR, reduce during training via callback
initial_learning_rate = 1e-3
optimizer = keras.optimizers.Adam(learning_rate=initial_learning_rate)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Verify output layer
print(f"\n{'='*60}")
print(f"✓ Model output shape: {model.output_shape}")
print(f"✓ Expected: (None, {num_classes})")
print(f"✓ Output units match num_classes: {model.layers[-1].units == num_classes}")
print(f"✓ Total trainable parameters: {model.count_params():,}")
print(f"{'='*60}")

In [ ]:
# STEP 8: CONFIGURE TRAINING CALLBACKS
os.makedirs(OUTPUT_DIR, exist_ok=True)

BEST_MODEL_PATH = os.path.join(OUTPUT_DIR, "best_model.keras")

# Checkpoint callback - saves best model based on validation accuracy
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    BEST_MODEL_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Early stopping - stop training if no improvement
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,  # Increased patience for better convergence
    restore_best_weights=True,
    mode='max',
    verbose=1
)

# Learning rate reduction - reduce LR when plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Reduce LR by half
    patience=5,   # Wait 5 epochs before reducing
    min_lr=1e-7,  # Don't reduce below this value
    mode='min',
    verbose=1
)

# TensorBoard callback for visualization in Colab
tensorboard_cb = keras.callbacks.TensorBoard(
    log_dir=os.path.join(OUTPUT_DIR, 'logs'),
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

# Custom callback to clear output and show progress (Colab-friendly)
class ColabProgressCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f"Epoch {epoch+1}: " +
              f"loss={logs.get('loss', 0):.4f}, " +
              f"accuracy={logs.get('accuracy', 0):.4f}, " +
              f"val_loss={logs.get('val_loss', 0):.4f}, " +
              f"val_accuracy={logs.get('val_accuracy', 0):.4f}")

# Adjust batch size based on available hardware
if tf.config.list_physical_devices('GPU'):
    BATCH_SIZE = 64  # GPU can handle larger batches
else:
    BATCH_SIZE = 32  # CPU works better with smaller batches

EPOCHS = 100

print(f"Training configuration:")
print(f"  • Batch size: {BATCH_SIZE} ({'GPU' if tf.config.list_physical_devices('GPU') else 'CPU'} optimized)")
print(f"  • Max epochs: {EPOCHS}")
print(f"  • Early stopping patience: 10 epochs")
print(f"  • Learning rate: {initial_learning_rate} (with reduction on plateau)")
print(f"  • Callbacks: Checkpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard")

print(f"\n✓ Best model will be saved to: {BEST_MODEL_PATH}")
print(f"✓ TensorBoard logs: {os.path.join(OUTPUT_DIR, 'logs')}")
print(f"\nTo view TensorBoard in Colab, run:")
print(f"  %load_ext tensorboard")
print(f"  %tensorboard --logdir {os.path.join(OUTPUT_DIR, 'logs')}")

In [ ]:
# STEP 9: TRAIN MODEL
print("\nStarting training...")
print("=" * 60)

# Combine callbacks
callbacks_list = [
    checkpoint_cb, 
    early_stopping, 
    reduce_lr, 
    tensorboard_cb,
    ColabProgressCallback()
]

# Train model with verbose=2 for cleaner Colab output
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    verbose=2  # Less verbose, better for Colab
)

print("\n" + "=" * 60)
print("✓ Training complete!")
print("=" * 60)

# Print training summary
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"\nFinal Training Metrics:")
print(f"  • Train Loss: {final_train_loss:.4f}")
print(f"  • Train Accuracy: {final_train_acc*100:.2f}%")
print(f"  • Val Loss: {final_val_loss:.4f}")
print(f"  • Val Accuracy: {final_val_acc*100:.2f}%")

# Check for overfitting
if final_train_acc - final_val_acc > 0.1:
    print(f"\n⚠ Warning: Possible overfitting detected!")
    print(f"  Train-Val accuracy gap: {(final_train_acc - final_val_acc)*100:.2f}%")
else:
    print(f"\n✓ Model generalization looks good!")

In [ ]:
# 10. EVALUATE MODEL
print("\nEvaluating model on validation set...")
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")

# Sample predictions check with confidence scores
print("\n" + "=" * 60)
print("Sample Predictions (first 10):")
print("=" * 60)

sample_preds = model.predict(X_val[:10], verbose=0)
print(f"Prediction shape: {sample_preds.shape} (Expected: (10, {num_classes}))")
assert sample_preds.shape == (10, num_classes), "Output shape mismatch!"

for i in range(10):
    pred_class = np.argmax(sample_preds[i])
    pred_label = label_encoder.classes_[pred_class]
    true_label = label_encoder.classes_[y_val[i]]
    confidence = sample_preds[i][pred_class] * 100
    
    # Color code: green for correct, red for incorrect
    status = "✓" if pred_label == true_label else "✗"
    print(f"{i+1:2d}. True: {true_label:3s} | Pred: {pred_label:3s} ({confidence:5.1f}%) {status}")

# Calculate confusion matrix for detailed analysis
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

print("\n" + "=" * 60)
print("Generating Classification Report...")
print("=" * 60)

# Predict all validation samples
all_preds = model.predict(X_val, verbose=0)
pred_classes = np.argmax(all_preds, axis=1)

# Classification report
report = classification_report(
    y_val, 
    pred_classes, 
    target_names=label_encoder.classes_,
    digits=3
)
print(report)

# Confusion matrix visualization
print("\nGenerating Confusion Matrix...")
cm = confusion_matrix(y_val, pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix - ASL Classifier')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

print("✓ Evaluation complete!")

In [ ]:
# 11. PLOT TRAINING HISTORY
print("\nGenerating training history plots...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Accuracy over epochs
axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Accuracy', fontsize=12)
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Loss over epochs
axes[0, 1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Loss', fontsize=12)
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Learning rate over epochs (if ReduceLROnPlateau was triggered)
if 'lr' in history.history:
    axes[1, 0].plot(history.history['lr'], linewidth=2, color='orange')
    axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch', fontsize=12)
    axes[1, 0].set_ylabel('Learning Rate', fontsize=12)
    axes[1, 0].set_yscale('log')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'Learning Rate history not available', 
                    ha='center', va='center', fontsize=12)
    axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')

# Plot 4: Per-class accuracy (if available)
# Show final validation accuracy per class
# Calculate accuracy for each class (handle empty classes)
per_class_acc = []
class_labels_present = []

for i, class_label in enumerate(label_encoder.classes_):
    mask = y_val == i
    if mask.sum() > 0:  # Only include classes present in validation set
        acc = np.mean(pred_classes[mask] == i)
        per_class_acc.append(acc)
        class_labels_present.append(class_label)

# Plot only classes present in validation set
x_positions = range(len(class_labels_present))
axes[1, 1].bar(x_positions, per_class_acc, color='skyblue', edgecolor='navy')
axes[1, 1].set_title('Per-Class Validation Accuracy', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Class', fontsize=12)
axes[1, 1].set_ylabel('Accuracy', fontsize=12)
axes[1, 1].set_xticks(x_positions)
axes[1, 1].set_xticklabels(class_labels_present, rotation=45, ha='right')
axes[1, 1].grid(True, alpha=0.3, axis='y')
axes[1, 1].set_ylim([0, 1.0])

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history.png'), dpi=150, bbox_inches='tight')
print(f"✓ Training plots saved to: {os.path.join(OUTPUT_DIR, 'training_history.png')}")
plt.show()

# Print summary statistics
print("\n" + "=" * 60)
print("Training Summary Statistics:")
print("=" * 60)
print(f"Total epochs trained: {len(history.history['loss'])}")
print(f"Best validation accuracy: {max(history.history['val_accuracy'])*100:.2f}%")
print(f"Best validation loss: {min(history.history['val_loss']):.4f}")
print(f"Final learning rate: {history.history.get('lr', [initial_learning_rate])[-1]:.2e}")
print("=" * 60)

In [ ]:
# STEP 12: SAVE MODEL AND LABELS
# Load best model from training
best_model = keras.models.load_model(BEST_MODEL_PATH)
print(f"✓ Loaded best model from: {BEST_MODEL_PATH}")

# Save labels.json (array format)
labels_path = os.path.join(OUTPUT_DIR, "labels.json")
with open(labels_path, 'w') as f:
    json.dump(label_encoder.classes_.tolist(), f, indent=2)
print(f"✓ Labels saved: {labels_path}")

# Verify labels.json content
with open(labels_path, 'r') as f:
    saved_labels = json.load(f)
print(f"\n✓ Verification: {len(saved_labels)} classes")
print(f"  Labels: {saved_labels}")

# Verify label count matches the encoder (use actual encoder classes count)
actual_num_classes = len(label_encoder.classes_)
if len(saved_labels) != actual_num_classes:
    print(f"⚠ Warning: saved_labels ({len(saved_labels)}) != label_encoder.classes_ ({actual_num_classes})")
    print(f"  This should not happen. Re-saving from encoder...")
    with open(labels_path, 'w') as f:
        json.dump(label_encoder.classes_.tolist(), f, indent=2)
    print(f"✓ Labels re-saved from encoder")

assert 'Train' not in saved_labels and 'Test' not in saved_labels, "Invalid labels detected!"
print(f"✓ Labels are correct! ({len(saved_labels)} classes)")

In [ ]:
# STEP 13: CONVERT TO TENSORFLOW.JS
import sys
import shutil

tfjs_output_dir = os.path.join(OUTPUT_DIR, "tfjs_model")

# Clean up old TFJS directory
if os.path.exists(tfjs_output_dir):
    shutil.rmtree(tfjs_output_dir)
    print('✓ Cleaned old TFJS directory')

os.makedirs(tfjs_output_dir, exist_ok=True)

print('\nConverting to TensorFlow.js format...')
print('This may take 1-2 minutes...\n')

try:
    # Upgrade tensorflowjs to a version compatible with NumPy 1.26+
    print('⚠ Upgrading TensorFlow.js to latest version...')
    !pip install -q --upgrade tensorflowjs
    print('✓ TensorFlow.js upgraded\n')
    
    # Use Python API instead of command-line tool to avoid NumPy compatibility issues
    import tensorflowjs as tfjs
    
    print('Using TensorFlow.js Python API for conversion...')
    
    # Convert using Python API - updated for latest tensorflowjs version
    # The newer API has simplified parameters
    tfjs.converters.save_keras_model(
        best_model,
        tfjs_output_dir
    )
    
    print('✓ TensorFlow.js conversion complete!')
    
    # Copy labels.json to tfjs directory
    tfjs_labels_path = os.path.join(tfjs_output_dir, "labels.json")
    shutil.copy(labels_path, tfjs_labels_path)
    print(f"✓ Labels copied to: {tfjs_labels_path}")
    
    # List generated files with sizes
    print(f'\n📦 Generated TFJS artifacts:')
    tfjs_files = sorted(os.listdir(tfjs_output_dir))
    total_size = 0
    
    for item in tfjs_files:
        item_path = os.path.join(tfjs_output_dir, item)
        size_mb = os.path.getsize(item_path) / (1024 * 1024)
        total_size += size_mb
        print(f'   • {item:<30} {size_mb:>8.2f} MB')
    
    print(f'\n   Total TFJS model size: {total_size:.2f} MB')
    
    # Create usage example file for web deployment
    usage_example = f"""
// TensorFlow.js Model Usage Example
// Generated: {os.path.basename(OUTPUT_DIR)}

// Load the model
const model = await tf.loadGraphModel('path/to/tfjs_model/model.json');

// Load labels
const response = await fetch('path/to/tfjs_model/labels.json');
const labels = await response.json();

// Make prediction
const inputTensor = tf.tensor2d([[...63 landmark features...]]); // Shape: [1, 63]
const prediction = model.predict(inputTensor);
const classIndex = prediction.argMax(-1).dataSync()[0];
const predictedLabel = labels[classIndex];

console.log('Predicted class:', predictedLabel);
"""
    
    usage_path = os.path.join(tfjs_output_dir, "usage_example.js")
    with open(usage_path, 'w') as f:
        f.write(usage_example)
    print(f'\n✓ Usage example saved to: {usage_path}')
    
except Exception as e:
    print(f'❌ TensorFlow.js conversion failed!')
    print(f'Error: {type(e).__name__}: {e}')
    import traceback
    traceback.print_exc()
    raise

In [ ]:
# STEP 14: FINAL VERIFICATION - Critical checks before deployment
print("Final Deployment Verification:")
print("="*60)

# Check 1: Model input shape
input_shape = best_model.input_shape
print(f"✓ Model input shape: {input_shape} (Expected: (None, 63))")
assert input_shape == (None, 63), "Input shape mismatch!"

# Check 2: Model output shape
output_shape = best_model.output_shape
actual_num_classes = len(label_encoder.classes_)
print(f"✓ Model output shape: {output_shape}")
print(f"  Note: Model has {output_shape[1]} outputs, but only {actual_num_classes} classes have labels")

# Check if there's a mismatch (one class may have had no valid samples)
if output_shape[1] != actual_num_classes:
    print(f"⚠ WARNING: Model output ({output_shape[1]}) != Label count ({actual_num_classes})")
    print(f"  This likely means one class had no valid landmark extractions")
    print(f"  The model will still work, but output index {actual_num_classes} will be unused")
    # This is acceptable - the extra output won't be used
else:
    assert output_shape == (None, actual_num_classes), "Output shape mismatch!"

# Check 3: Labels file exists and is correct
with open(tfjs_labels_path, 'r') as f:
    tfjs_labels = json.load(f)
print(f"✓ Labels in TFJS: {len(tfjs_labels)} classes")
print(f"  {tfjs_labels}")
assert len(tfjs_labels) == actual_num_classes, "Label count mismatch with encoder!"
assert isinstance(tfjs_labels, list), "Labels must be an array!"

# Check 4: No Train/Test labels
assert 'Train' not in tfjs_labels and 'Test' not in tfjs_labels, \
    "CRITICAL: Train/Test labels still present!"
print(f"✓ No 'Train'/'Test' labels detected")

# Check 5: Files exist
tfjs_model_json = os.path.join(tfjs_output_dir, "model.json")
assert os.path.exists(tfjs_model_json), "model.json not found!"
print(f"✓ model.json exists")

print("="*60)
print("\n🎉 MODEL READY FOR DEPLOYMENT!")
print(f"\n📂 All files saved to: {OUTPUT_DIR}")

# Check for output size mismatch
if best_model.output_shape[1] != len(label_encoder.classes_):
    print(f"\n⚠ IMPORTANT NOTE:")
    print(f"  Model has {best_model.output_shape[1]} outputs, but only {len(label_encoder.classes_)} labels")
    print(f"  One dataset class had no valid landmark extractions")
    print(f"  The model works fine - just ignore the unused output index")

print(f"\nNext steps:")
print(f"1. Download '{OUTPUT_DIR}' folder from Google Drive")
print(f"2. Copy tfjs_model/ to your web app")
print(f"3. Load: const model = await tf.loadGraphModel('path/to/model.json');")
print(f"4. Load: const labels = await fetch('path/to/labels.json').then(r=>r.json());")
print(f"5. Ensure web app extracts 63 landmarks and centers at wrist")
print(f"6. Get prediction: const classIdx = tf.argMax(prediction, -1); // Use labels[classIdx]")

In [ ]:
# STEP 15: TEST END-TO-END PREDICTION & GENERATE TRAINING REPORT
import datetime

# Test prediction
test_idx = np.random.randint(0, len(X_val))
test_sample = X_val[test_idx:test_idx+1]
test_label = label_encoder.classes_[y_val[test_idx]]

prediction = best_model.predict(test_sample, verbose=0)
pred_class = np.argmax(prediction[0])
pred_label = label_encoder.classes_[pred_class]
confidence = prediction[0][pred_class] * 100

print(f"End-to-End Test:")
print(f"  Input: {test_sample.shape}, Output: {prediction.shape}")
print(f"  True: {test_label} | Predicted: {pred_label} ({confidence:.1f}%)")
print(f"  Match: {'✓' if test_label == pred_label else '✗'}")

# Top 3 predictions
top3_indices = np.argsort(prediction[0])[-3:][::-1]
print(f"\n  Top 3 predictions:")
for i, idx in enumerate(top3_indices, 1):
    label = label_encoder.classes_[idx]
    conf = prediction[0][idx] * 100
    print(f"    {i}. {label}: {conf:.1f}%")

# Generate comprehensive training report
print(f"\n{'='*60}")
print('GENERATING TRAINING REPORT...')
print(f"{'='*60}")

report_content = f"""
# ASL Sign Language Model Training Report
Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Environment Information
- TensorFlow Version: {tf.__version__}
- MediaPipe Version: {mp.__version__}
- Python Version: {sys.version}
- GPU Available: {len(tf.config.list_physical_devices('GPU'))} GPU(s)
- GPU Devices: {tf.config.list_physical_devices('GPU')}

## Dataset Information
- Total Classes: {num_classes}
- Classes: {', '.join(label_encoder.classes_)}
- Total Images Processed: {total_images:,}
- Successful Extractions: {len(X_data):,}
- Skipped Images: {skipped_count:,} ({skip_ratio:.1%})
- Feature Dimensions: {X_data.shape[1]}

## Data Split
- Training Samples: {len(X_train):,} ({len(X_train)/len(X_data)*100:.1f}%)
- Validation Samples: {len(X_val):,} ({len(X_val)/len(X_data)*100:.1f}%)
- Augmentation Applied: {ENABLE_AUGMENTATION}

## Model Architecture
- Model Type: Sequential MLP Classifier
- Input Shape: ({X_data.shape[1]},)
- Total Parameters: {model.count_params():,}
- Trainable Parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}
- Layer Configuration:
  * Dense(256) + BatchNorm + Dropout(0.3)
  * Dense(128) + BatchNorm + Dropout(0.2)
  * Dense(64) + Dropout(0.1)
  * Dense({num_classes}, softmax)

## Training Configuration
- Optimizer: Adam
- Initial Learning Rate: {initial_learning_rate}
- Batch Size: {BATCH_SIZE}
- Max Epochs: {EPOCHS}
- Epochs Trained: {len(history.history['loss'])}
- Early Stopping Patience: 10
- Learning Rate Reduction: ReduceLROnPlateau (factor=0.5, patience=5)

## Performance Metrics
### Training Results
- Final Train Loss: {final_train_loss:.4f}
- Final Train Accuracy: {final_train_acc*100:.2f}%
- Final Val Loss: {final_val_loss:.4f}
- Final Val Accuracy: {final_val_acc*100:.2f}%

### Best Results
- Best Validation Accuracy: {max(history.history['val_accuracy'])*100:.2f}%
- Best Validation Loss: {min(history.history['val_loss']):.4f}

### Model Generalization
- Train-Val Accuracy Gap: {(final_train_acc - final_val_acc)*100:.2f}%
- Overfitting Status: {'Possible overfitting' if final_train_acc - final_val_acc > 0.1 else 'Good generalization'}

## Output Files
- Best Model: {BEST_MODEL_PATH}
- Labels File: {labels_path}
- TFJS Model: {tfjs_output_dir}
- Training Plots: {os.path.join(OUTPUT_DIR, 'training_history.png')}
- TensorBoard Logs: {os.path.join(OUTPUT_DIR, 'logs')}

## Deployment Instructions
1. Download the entire output directory: {OUTPUT_DIR}
2. For web deployment:
   - Use model.json from tfjs_model/
   - Load labels from tfjs_model/labels.json
   - Input: 63 MediaPipe hand landmarks (wrist-centered)
   - Output: Probability distribution over {num_classes} classes
3. Recommended: Implement smoothing (5-frame majority voting) for real-time predictions

## Notes
- Model uses wrist-centered MediaPipe landmarks (21 landmarks × 3 coordinates = 63 features)
- Labels format: Array of strings (not dictionary)
- Model expects normalized landmark coordinates (0-1 range)
- Recommended inference: Extract landmarks → Center at wrist → Predict

---
Report generated by ASL Model Training Pipeline
"""

# Save report
report_path = os.path.join(OUTPUT_DIR, 'training_report.md')
with open(report_path, 'w') as f:
    f.write(report_content)

print(f"\n✓ Training report saved to: {report_path}")

print(f"\n{'='*60}")
print('TRAINING COMPLETE - ALL ARTIFACTS SAVED!')
print(f"{'='*60}")
print(f"\n📊 Summary:")
print(f"  • Model: {BEST_MODEL_PATH}")
print(f"  • Labels: {labels_path}")
print(f"  • TFJS Model: {tfjs_output_dir}")
print(f"  • Training Report: {report_path}")
print(f"  • Classes: {num_classes}")
print(f"  • Training samples: {len(X_train):,}")
print(f"  • Validation samples: {len(X_val):,}")
print(f"  • Best Val Accuracy: {max(history.history['val_accuracy'])*100:.2f}%")
print(f"\n✅ All artifacts saved to Google Drive!")
print(f"📥 Download from: {OUTPUT_DIR}")
print(f"\n💡 Next Steps:")
print(f"  1. Review training_report.md for detailed metrics")
print(f"  2. Check confusion matrix for per-class performance")
print(f"  3. Download all files from Google Drive")
print(f"  4. Deploy tfjs_model/ to your web application")
print(f"  5. Use usage_example.js as reference for integration")